In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from pandas import DatetimeIndex
from pandas.api.types import is_numeric_dtype
from sklearn.preprocessing import LabelEncoder
from IPython.display import display, HTML 

pd.set_option('display.max_columns', 500)

%pip install openpyxl
%matplotlib inline


Note: you may need to restart the kernel to use updated packages.


In [3]:
df = pd.read_csv('diabetes_data.csv') 

# Duplicates Drop

In [14]:
print(df.duplicated().sum())
print(df.drop_duplicates())
df['Diabetes'].value_counts()


6672
        Age  Sex  HighChol  CholCheck   BMI  Smoker  HeartDiseaseorAttack  \
0       4.0  1.0       0.0        1.0  26.0     0.0                   0.0   
1      12.0  1.0       1.0        1.0  26.0     1.0                   0.0   
2      13.0  1.0       0.0        1.0  26.0     0.0                   0.0   
3      11.0  1.0       1.0        1.0  28.0     1.0                   0.0   
4       8.0  0.0       0.0        1.0  29.0     1.0                   0.0   
...     ...  ...       ...        ...   ...     ...                   ...   
70687   6.0  0.0       1.0        1.0  37.0     0.0                   0.0   
70688  10.0  1.0       1.0        1.0  29.0     1.0                   1.0   
70689  13.0  0.0       1.0        1.0  25.0     0.0                   1.0   
70690  11.0  0.0       1.0        1.0  18.0     0.0                   0.0   
70691   9.0  0.0       1.0        1.0  25.0     0.0                   1.0   

       PhysActivity  Fruits  Veggies  HvyAlcoholConsump  GenHlth  Ment

Diabetes
0.0    35346
1.0    35346
Name: count, dtype: int64